## 问题定义

本问题为求解高热导率差异的两个物体间的热传导。其定义如下图：

![problem](./resource/problem.png)


其中中间的小物体为热源，热导率为$385W/mK$，外围为低热导率的物体，其热导率为$0.0261W/mk$。


### 求解目标

要求求解上述求解区域的稳态解

## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

In [3]:
import torch
import numpy as np
from sympy import Symbol, Eq, Or, And

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry import Bounds
from modulus.sym.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.sym.eq.pdes.navier_stokes import GradNormal
from modulus.sym.eq.pdes.diffusion import Diffusion, DiffusionInterface
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)
from modulus.sym.models.activation import Activation
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.models.modified_fourier_net import ModifiedFourierNetArch

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 100000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 1000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch

### 定义必要组件

#### Geo

In [5]:
channel_origin = (-2.5, -0.5)  # channel原点（左下角）
channel_dim = (5.0, 1.0)  # 长宽
heat_sink_base_origin = (-1.0, -0.5)  # heat sink原点（左下角）
heat_sink_base_dim = (1.0, 0.2)  # 长宽
fin_origin = heat_sink_base_origin  # fin原点（左下角）
fin_dim = (1.0, 0.6)  # 长宽
total_fins = 1  # fin总数
box_origin = (-1.1, -0.5)  # 相较于sink的位置，向入口移动0.1个单位
box_dim = (1.2, 1.0)  # 总长比sink长0.2个单位
source_origin = (-0.7, -0.5)  # 热源原点（左下角）
source_dim = (0.4, 0.0)  # 长宽
source_length = 0.4

inlet_temp = 25.0  # 入口温度
conductivity_I = 0.01  # 周围环境热导率
conductivity_II = 100.0  # sink热导率
source_grad = 0.025  # 热源的温度梯度

In [6]:
# define geometry
# channel
channel = Channel2D(
    channel_origin,
    (channel_origin[0] + channel_dim[0], channel_origin[1] + channel_dim[1]),
)  # 整个channel
# heat sink
heat_sink_base = Rectangle(
    heat_sink_base_origin,
    (
        heat_sink_base_origin[0] + heat_sink_base_dim[0],  # base of heat sink
        heat_sink_base_origin[1] + heat_sink_base_dim[1],
    ),
)  # heat sink
fin_center = (fin_origin[0] + fin_dim[0] / 2, fin_origin[1] + fin_dim[1] / 2)
fin = Rectangle(
    fin_origin, (fin_origin[0] + fin_dim[0], fin_origin[1] + fin_dim[1])
)  # fin
chip2d = heat_sink_base + fin  # 芯片位置

# entire geometry
geo = channel - chip2d  # 非芯片位置

# low and high resultion geo away and near the heat sink
box = Rectangle(
    box_origin,
    (box_origin[0] + box_dim[0], box_origin[1] + box_dim[1]),  # base of heat sink
)

lr_geo = geo - box  # 远离芯片的区域
hr_geo = geo & box  # 靠近芯片的区域

lr_bounds_x = (channel_origin[0], channel_origin[0] + channel_dim[0])
lr_bounds_y = (channel_origin[1], channel_origin[1] + channel_dim[1])

hr_bounds_x = (box_origin[0], box_origin[0] + box_dim[0])
hr_bounds_y = (box_origin[1], box_origin[1] + box_dim[1])

# inlet and outlet
inlet = Line(
    channel_origin, (channel_origin[0], channel_origin[1] + channel_dim[1]), -1
)  # 入口线
outlet = Line(
    (channel_origin[0] + channel_dim[0], channel_origin[1]),
    (channel_origin[0] + channel_dim[0], channel_origin[1] + channel_dim[1]),
    1,
)  # 出口线

#### PDE

In [7]:
d_solid_I = Diffusion(T="theta_I", D=1.0, dim=2, time=False)  # 一号区域内的Diffusion，这里比较奇怪 - D设置为1，和前置的问题定义不符
d_solid_II = Diffusion(T="theta_II", D=1.0, dim=2, time=False)  # 二号区域内的Diffusion，这里比较奇怪 - D设置为1，和前置的问题定义不符

interface = DiffusionInterface(
        "theta_I", "theta_II", conductivity_I, conductivity_II, dim=2, time=False
    )  # 界面

gn_solid_I = GradNormal("theta_I", dim=2, time=False)  # 用于实现梯度边界条件
gn_solid_II = GradNormal("theta_II", dim=2, time=False)  # 用于实现梯度边界条件

#### Model

In [8]:
# 定义为FourierNet
# 输入为空间坐标
# 输出为预测的温度
solid_I_net = ModifiedFourierNetArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("theta_I_star")],
    layer_size=128,
    frequencies=("gaussian", 0.2, 64),
    activation_fn=Activation.TANH,
)
solid_II_net = ModifiedFourierNetArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("theta_II_star")],
    layer_size=128,
    frequencies=("gaussian", 0.2, 64),
    activation_fn=Activation.TANH,
)

#### Node

In [9]:
nodes = (
    d_solid_I.make_nodes()  # 一号区域Diffusion Model
    + d_solid_II.make_nodes()  # 二号区域Diffusion Model
    + interface.make_nodes()  # 界面Model
    + gn_solid_I.make_nodes()  # Neumann边界
    + gn_solid_II.make_nodes()  # Neumann边界
    + [
        Node.from_sympy(100 * Symbol("theta_I_star") + 25.0, "theta_I")  # 归一化温度，预先估计了温度区间
    ]  # Normalize the outputs
    + [
        Node.from_sympy(Symbol("theta_II_star") + 200.0, "theta_II")  # 归一化温度，预先估计了温度区间
    ]  # Normalize the outputs
    + [solid_I_net.make_node(name="solid_I_network")]
    + [solid_II_net.make_node(name="solid_II_network")]
)

x, y = Symbol("x"), Symbol("y")

#### Domain

In [10]:
domain = Domain()

#### 入口条件

In [11]:
inlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=inlet,
    outvar={"theta_I": inlet_temp},
    lambda_weighting={"theta_I": 10.0},
    batch_size=cfg.batch_size.inlet,
)
domain.add_constraint(inlet, "inlet")  # 入口温度监督

#### 出口条件

In [12]:
outlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=outlet,
    outvar={"normal_gradient_theta_I": 0},  # 基于gn_solid_I获得
    batch_size=cfg.batch_size.outlet,
)
domain.add_constraint(outlet, "outlet")  # 出口的温度梯度为0

#### channel的绝缘边缘

In [13]:
# channel walls insulating
def walls_criteria(invar, params):
    sdf = chip2d.sdf(invar, params)  
    return np.less(sdf["sdf"], -1e-5)  # chip范围外

walls = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=channel,
    outvar={"normal_gradient_theta_I": 0},
    batch_size=cfg.batch_size.walls,
    criteria=walls_criteria,
)
domain.add_constraint(walls, "channel_walls")  

#### pde

In [14]:
# 区域I PDE （远离chip区域）
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=lr_geo,
    outvar={"diffusion_theta_I": 0},
    batch_size=cfg.batch_size.interior_lr,
    lambda_weighting={"diffusion_theta_I": 1.0},
)
domain.add_constraint(interior, "solid_I_interior_lr")

# 区域I PDE （靠近chip区域）
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=hr_geo,
    outvar={"diffusion_theta_I": 0},
    batch_size=cfg.batch_size.interior_hr,
    lambda_weighting={"diffusion_theta_I": 1.0},
)
domain.add_constraint(interior, "solid_I_interior_hr")


# 区域II PDE
interiorS = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"diffusion_theta_II": 0},
    batch_size=cfg.batch_size.interiorS,
    lambda_weighting={"diffusion_theta_II": 1.0},
)
domain.add_constraint(interiorS, "solid_II_interior")

#### 界面

In [15]:
def interface_criteria(invar, params):
    sdf = channel.sdf(invar, params)  # 面向channel的 SDF
    return np.greater(sdf["sdf"], 0)  # channel范围内

interface = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,  # 采样chip的boundary，结合interface_criteria的定义即在界面位置采样
    outvar={
        "diffusion_interface_dirichlet_theta_I_theta_II": 0,
        "diffusion_interface_neumann_theta_I_theta_II": 0,
    },
    batch_size=cfg.batch_size.interface,
    lambda_weighting={
        "diffusion_interface_dirichlet_theta_I_theta_II": 10,
        "diffusion_interface_neumann_theta_I_theta_II": 1,
    },
    criteria=interface_criteria,
)
domain.add_constraint(interface, name="interface")

####  热源

In [16]:
# 热源区间
heat_source = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"normal_gradient_theta_II": source_grad},
    batch_size=cfg.batch_size.heat_source,
    lambda_weighting={"normal_gradient_theta_II": 1000},
    criteria=(
        Eq(y, source_origin[1])
        & (x >= source_origin[0])
        & (x <= (source_origin[0] + source_dim[0]))
    ),
)
domain.add_constraint(heat_source, name="heat_source")

#### chip wall

In [17]:
chip_walls = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"normal_gradient_theta_II": 0},
    batch_size=cfg.batch_size.chip_walls,
    # lambda_weighting={"normal_gradient_theta_II": 1000},
    criteria=(
        Eq(y, source_origin[1])
        & ((x < source_origin[0]) | (x > (source_origin[0] + source_dim[0])))
    ),  # 靠近边界的外侧
)
domain.add_constraint(chip_walls, name="chip_walls")

#### 验证器以及其他必要组件

In [18]:
monitor = PointwiseMonitor(
    chip2d.sample_boundary(10000, criteria=Eq(y, source_origin[1])),
    output_names=["theta_II"],
    metrics={
        "peak_temp": lambda var: torch.max(var["theta_II"]),
    },
    nodes=nodes,  # 监督峰值温度
)
domain.add_monitor(monitor)


In [19]:
# 区域I GT
file_path = "openfoam/2d_solid_solid_D1.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"Points:0": "x", "Points:1": "y", "Temperature": "theta_I"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += channel_origin[0]  # normalize pos
    openfoam_var["y"] += channel_origin[1]
    openfoam_invar_solid_I_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_solid_I_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["theta_I"]
    }
    openfoam_validator_solid_I = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_solid_I_numpy,
        true_outvar=openfoam_outvar_solid_I_numpy,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator_solid_I)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

In [20]:
# 区域II GT
file_path = "openfoam/2d_solid_solid_D2.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"Points:0": "x", "Points:1": "y", "Temperature": "theta_II"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += channel_origin[0]  # normalize pos
    openfoam_var["y"] += channel_origin[1]
    openfoam_invar_solid_II_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_solid_II_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["theta_II"]
    }
    openfoam_validator_solid_II = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_solid_II_numpy,
        true_outvar=openfoam_outvar_solid_II_numpy,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator_solid_II)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [21]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [22]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [23]:
slv.solve()

2024-03-01 08:03:13,346 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 08:03:13,354 - INFO - attempting to restore from: /workspace/15_2D_ChipSolidSolid_HeatTransfer/outputs
2024-03-01 08:03:13,355 - WARNING - optimizer checkpoint not found
2024-03-01 08:03:13,356 - WARNING - model solid_II_network.0.pth not found
2024-03-01 08:03:13,356 - WARNING - model solid_I_network.0.pth not found
2024-03-01 08:03:14,810 - INFO - [step:          0] record constraint batch time:  2.585e-01s
2024-03-01 08:03:17,820 - INFO - [step:          0] record validators time:  3.006e+00s
2024-03-01 08:03:17,911 - INFO - [step:          0] record monitor time:  6.758e-02s
2024-03-01 08:03:17,942 - INFO - [step:          0] saved checkpoint to /workspace/15_2D_ChipSolidSolid_HeatTransfer/outputs
2024-03-01 08:03:17,945 - INFO - [step:          0] loss:  1.654e+06
2024-03-01 08:03:23,833 - INFO - Attempti

2024-03-01 08:07:33,488 - INFO - [step:       5300] loss:  1.083e+02, time/iteration:  4.266e+01 ms
2024-03-01 08:07:37,611 - INFO - [step:       5400] loss:  1.882e+02, time/iteration:  4.121e+01 ms
2024-03-01 08:07:41,866 - INFO - [step:       5500] loss:  4.772e+01, time/iteration:  4.254e+01 ms
2024-03-01 08:07:46,013 - INFO - [step:       5600] loss:  1.288e+02, time/iteration:  4.146e+01 ms
2024-03-01 08:07:50,068 - INFO - [step:       5700] loss:  7.874e+01, time/iteration:  4.054e+01 ms
2024-03-01 08:07:54,066 - INFO - [step:       5800] loss:  3.551e+02, time/iteration:  3.996e+01 ms
2024-03-01 08:07:58,229 - INFO - [step:       5900] loss:  2.683e+02, time/iteration:  4.162e+01 ms
2024-03-01 08:08:03,412 - INFO - [step:       6000] record constraint batch time:  2.689e-01s
2024-03-01 08:08:06,409 - INFO - [step:       6000] record validators time:  2.996e+00s
2024-03-01 08:08:06,476 - INFO - [step:       6000] record monitor time:  2.835e-02s
2024-03-01 08:08:06,509 - INFO - 

2024-03-01 08:12:02,552 - INFO - [step:      11200] loss:  8.739e+01, time/iteration:  3.915e+01 ms
2024-03-01 08:12:06,580 - INFO - [step:      11300] loss:  5.912e+01, time/iteration:  4.026e+01 ms
2024-03-01 08:12:09,677 - INFO - [step:      11400] loss:  6.161e+01, time/iteration:  3.095e+01 ms
2024-03-01 08:12:13,615 - INFO - [step:      11500] loss:  4.587e+01, time/iteration:  3.936e+01 ms
2024-03-01 08:12:17,091 - INFO - [step:      11600] loss:  7.528e+01, time/iteration:  3.475e+01 ms
2024-03-01 08:12:20,522 - INFO - [step:      11700] loss:  8.057e+01, time/iteration:  3.430e+01 ms
2024-03-01 08:12:23,292 - INFO - [step:      11800] loss:  4.903e+01, time/iteration:  2.768e+01 ms
2024-03-01 08:12:25,889 - INFO - [step:      11900] loss:  4.627e+01, time/iteration:  2.595e+01 ms
2024-03-01 08:12:29,383 - INFO - [step:      12000] record constraint batch time:  1.631e-01s
2024-03-01 08:12:32,353 - INFO - [step:      12000] record validators time:  2.968e+00s
2024-03-01 08:12:3

2024-03-01 08:16:22,106 - INFO - [step:      17100] loss:  1.595e+01, time/iteration:  4.134e+01 ms
2024-03-01 08:16:26,231 - INFO - [step:      17200] loss:  2.184e+01, time/iteration:  4.124e+01 ms
2024-03-01 08:16:30,360 - INFO - [step:      17300] loss:  5.462e+01, time/iteration:  4.128e+01 ms
2024-03-01 08:16:34,506 - INFO - [step:      17400] loss:  3.445e+01, time/iteration:  4.144e+01 ms
2024-03-01 08:16:38,620 - INFO - [step:      17500] loss:  2.416e+01, time/iteration:  4.113e+01 ms
2024-03-01 08:16:42,726 - INFO - [step:      17600] loss:  2.264e+01, time/iteration:  4.104e+01 ms
2024-03-01 08:16:46,793 - INFO - [step:      17700] loss:  3.958e+01, time/iteration:  4.065e+01 ms
2024-03-01 08:16:50,773 - INFO - [step:      17800] loss:  2.698e+01, time/iteration:  3.979e+01 ms
2024-03-01 08:16:54,797 - INFO - [step:      17900] loss:  2.013e+01, time/iteration:  4.023e+01 ms
2024-03-01 08:16:59,888 - INFO - [step:      18000] record constraint batch time:  2.469e-01s
2024-0

2024-03-01 08:20:46,933 - INFO - [step:      23000] loss:  1.353e+01, time/iteration:  8.486e+01 ms
2024-03-01 08:20:51,035 - INFO - [step:      23100] loss:  8.942e+00, time/iteration:  4.101e+01 ms
2024-03-01 08:20:55,127 - INFO - [step:      23200] loss:  1.339e+01, time/iteration:  4.091e+01 ms
2024-03-01 08:20:59,373 - INFO - [step:      23300] loss:  1.047e+01, time/iteration:  4.245e+01 ms
2024-03-01 08:21:03,465 - INFO - [step:      23400] loss:  2.553e+01, time/iteration:  4.091e+01 ms
2024-03-01 08:21:07,365 - INFO - [step:      23500] loss:  2.108e+01, time/iteration:  3.899e+01 ms
2024-03-01 08:21:11,244 - INFO - [step:      23600] loss:  1.217e+01, time/iteration:  3.877e+01 ms
2024-03-01 08:21:15,058 - INFO - [step:      23700] loss:  1.579e+01, time/iteration:  3.813e+01 ms
2024-03-01 08:21:19,309 - INFO - [step:      23800] loss:  6.336e+00, time/iteration:  4.250e+01 ms
2024-03-01 08:21:23,412 - INFO - [step:      23900] loss:  1.418e+01, time/iteration:  4.101e+01 ms


...

2024-03-01 09:01:07,813 - INFO - [step:      76500] loss:  2.004e+00, time/iteration:  4.126e+01 ms
2024-03-01 09:01:12,021 - INFO - [step:      76600] loss:  1.725e+00, time/iteration:  4.206e+01 ms
2024-03-01 09:01:16,215 - INFO - [step:      76700] loss:  1.304e+00, time/iteration:  4.193e+01 ms
2024-03-01 09:01:20,317 - INFO - [step:      76800] loss:  2.788e+00, time/iteration:  4.100e+01 ms
2024-03-01 09:01:24,399 - INFO - [step:      76900] loss:  1.084e+00, time/iteration:  4.081e+01 ms
2024-03-01 09:01:29,604 - INFO - [step:      77000] record constraint batch time:  2.434e-01s
2024-03-01 09:01:32,603 - INFO - [step:      77000] record validators time:  2.997e+00s
2024-03-01 09:01:32,692 - INFO - [step:      77000] record monitor time:  7.030e-02s
2024-03-01 09:01:32,726 - INFO - [step:      77000] saved checkpoint to /workspace/15_2D_ChipSolidSolid_HeatTransfer/outputs
2024-03-01 09:01:32,728 - INFO - [step:      77000] loss:  1.429e+00, time/iteration:  8.328e+01 ms
2024-03-

2024-03-01 09:05:29,040 - INFO - [step:      82400] loss:  7.445e-01, time/iteration:  4.275e+01 ms
2024-03-01 09:05:33,184 - INFO - [step:      82500] loss:  1.441e+00, time/iteration:  4.143e+01 ms
2024-03-01 09:05:37,632 - INFO - [step:      82600] loss:  9.852e-01, time/iteration:  4.447e+01 ms
2024-03-01 09:05:41,499 - INFO - [step:      82700] loss:  2.117e+00, time/iteration:  3.865e+01 ms
2024-03-01 09:05:45,564 - INFO - [step:      82800] loss:  4.258e+00, time/iteration:  4.064e+01 ms
2024-03-01 09:05:49,766 - INFO - [step:      82900] loss:  1.764e+00, time/iteration:  4.201e+01 ms
2024-03-01 09:05:55,218 - INFO - [step:      83000] record constraint batch time:  1.318e-01s
2024-03-01 09:05:58,695 - INFO - [step:      83000] record validators time:  3.472e+00s
2024-03-01 09:05:58,728 - INFO - [step:      83000] record monitor time:  7.330e-03s
2024-03-01 09:05:58,750 - INFO - [step:      83000] saved checkpoint to /workspace/15_2D_ChipSolidSolid_HeatTransfer/outputs
2024-03-

2024-03-01 09:09:59,884 - INFO - [step:      88300] loss:  1.315e+00, time/iteration:  3.961e+01 ms
2024-03-01 09:10:03,913 - INFO - [step:      88400] loss:  1.353e+00, time/iteration:  4.026e+01 ms
2024-03-01 09:10:08,156 - INFO - [step:      88500] loss:  1.236e+00, time/iteration:  4.241e+01 ms
2024-03-01 09:10:12,543 - INFO - [step:      88600] loss:  6.992e-01, time/iteration:  4.386e+01 ms
2024-03-01 09:10:16,640 - INFO - [step:      88700] loss:  6.368e-01, time/iteration:  4.096e+01 ms
2024-03-01 09:10:20,744 - INFO - [step:      88800] loss:  9.422e-01, time/iteration:  4.102e+01 ms
2024-03-01 09:10:24,857 - INFO - [step:      88900] loss:  8.922e-01, time/iteration:  4.112e+01 ms
2024-03-01 09:10:29,917 - INFO - [step:      89000] record constraint batch time:  2.298e-01s
2024-03-01 09:10:32,917 - INFO - [step:      89000] record validators time:  2.998e+00s
2024-03-01 09:10:33,001 - INFO - [step:      89000] record monitor time:  5.530e-02s
2024-03-01 09:10:33,019 - INFO - 

2024-03-01 09:14:28,865 - INFO - [step:      94200] loss:  1.095e+00, time/iteration:  4.373e+01 ms
2024-03-01 09:14:32,998 - INFO - [step:      94300] loss:  9.715e-01, time/iteration:  4.132e+01 ms
2024-03-01 09:14:37,187 - INFO - [step:      94400] loss:  1.096e+00, time/iteration:  4.188e+01 ms
2024-03-01 09:14:41,340 - INFO - [step:      94500] loss:  2.171e+00, time/iteration:  4.152e+01 ms
2024-03-01 09:14:45,482 - INFO - [step:      94600] loss:  1.277e+00, time/iteration:  4.141e+01 ms
2024-03-01 09:14:49,746 - INFO - [step:      94700] loss:  7.851e-01, time/iteration:  4.261e+01 ms
2024-03-01 09:14:54,010 - INFO - [step:      94800] loss:  8.229e-01, time/iteration:  4.262e+01 ms
2024-03-01 09:14:58,297 - INFO - [step:      94900] loss:  7.083e-01, time/iteration:  4.285e+01 ms
2024-03-01 09:15:03,555 - INFO - [step:      95000] record constraint batch time:  2.102e-01s
2024-03-01 09:15:06,599 - INFO - [step:      95000] record validators time:  3.042e+00s
2024-03-01 09:15:0

2024-03-01 09:18:58,051 - INFO - [step:     100000] reached maximum training steps, finished training!


### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![area_1](./outputs/validators/validator_theta_I.png)

![area_2](./outputs/validators/validator_2_theta_II.png)

In [24]:
import pandas as pd

In [25]:
peak_temp_df = pd.read_csv('./outputs/monitors/peak_temp.csv')

In [26]:
peak_temp_df

,step,peak_temp
0,0,200.54936
1,1000,199.74072
2,2000,199.32597
3,3000,199.01724
4,4000,198.81041
...,...,...
96,96000,186.41882
97,97000,186.39462
98,98000,186.37016
99,99000,186.34639
